# 部署你的第一个语言模型：本地或云端

> 指导文章：[06. 开始实践：部署你的第一个语言模型](https://github.com/Hoper-J/LLM-Guide-and-Demos-zh_CN/blob/master/Guide/06.%20开始实践：部署你的第一个语言模型.md)

在之前的章节中，我们已经了解了 Hugging Face 中 `AutoModel` 系列的不同类。现在，我们将使用一个参数量较小的模型，为你进行演示，并向你展示如何使用 FastAPI 和 Flask 将模型部署为 API 服务。

## 安装库

In [ ]:
!uv add transformers
!uv add sentencepiece

## 设置模型下载镜像

注意，需要在导入 transformers 等模块前进行设置才能起效。

In [ ]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

## 选择并加载模型

我们选择一个参数量较小的模型，如 `distilgpt2`，这是 GPT-2 的精简版本（或者说蒸馏），只有约 8820 万参数。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 指定模型名称
model_name = "distilgpt2"

# 加载 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 加载预训练模型
model = AutoModelForCausalLM.from_pretrained(model_name)

## 移动模型到合适的设备

如果你的计算机有 GPU，可将模型移动到 GPU，加快推理速度。如果你使用的是 Apple 芯片的 Mac，可以移动到 `mps` 上。

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() 
                      else "mps" if torch.backends.mps.is_available() 
                      else "cpu")
model.to(device)

## 进行推理

现在，我们可以使用模型进行文本生成。

In [ ]:
# 设置模型为评估模式
model.eval()

# 输入文本
input_text = "Hello GPT"

# 编码输入文本
inputs = tokenizer(input_text, return_tensors="pt")
inputs = {key: value.to(device) for key, value in inputs.items()}

# 生成文本
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=200,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=False
    )

# 解码生成的文本
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("模型生成的文本：")
print(generated_text)

## 部署模型为 API 服务（可选）

如果你希望将模型部署为一个 API 服务，供其他应用调用，可以使用 `fastapi` 框架。

### 使用 FastAPI 部署模型
#### 安装 `fastapi` 和 `uvicorn`

In [ ]:
!uv add fastapi uvicorn

#### 创建 API 服务

把下面这段代码保存到 `app_fastapi.py` 文件中（你也可以命名为其他的）

```python
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# 定义请求体的数据模型
class PromptRequest(BaseModel):
    prompt: str

app = FastAPI()

# 加载模型和分词器
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

@app.post("/generate")
def generate_text(request: PromptRequest):
    prompt = request.prompt
    if not prompt:
        raise HTTPException(status_code=400, detail="No prompt provided")

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=200,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return {"generated_text": generated_text}

```

然后在命令行执行以下命令：

```bash
uvicorn app_fastapi:app --host 0.0.0.0 --port 8000
```

你应该可以看到：
![image-20240914112627874](../Guide/assets/image-20240914131113829.png)

#### 通过 API 调用模型

现在，你可以访问 [http://localhost:8000/docs](http://localhost:8000/docs) 交互使用，或者通过发送 HTTP 请求来调用模型。

In [ ]:
import requests

response = requests.post(
    "http://localhost:8000/generate",
    json={"prompt": "Hello GPT"}
)
print(response.json())

### 使用 Flask 部署模型

#### 安装 Flask


In [ ]:
!uv add flask

#### 创建 API 服务

把下面这段代码保存到 `app_flask.py` 文件中（你也可以命名为其他的）

```python
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

app = Flask(__name__)

# 加载模型和分词器
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

@app.route('/generate', methods=['POST'])
def generate():
    prompt = request.json.get('prompt')
    if not prompt:
        return jsonify({'error': 'No prompt provided'}), 400

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=200,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return jsonify({'generated_text': generated_text})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)


```

然后在命令行执行以下命令：

```bash
python app_flask.py
```

你应该可以看到：
![image-20240914112627874](../Guide/assets/20240914143358.png)

#### 通过 API 调用模型

现在，你可以通过发送 HTTP 请求来调用模型。

In [ ]:
import requests

response = requests.post(
    "http://localhost:8000/generate",
    json={"prompt": "Hello GPT"}
)
print(response.json())